# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Markdown cell with the import command:
mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [15]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [16]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [17]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [18]:
# review the collections in our database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:", collections)

Collections in 'uk_food' database: ['establishments']


In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [20]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print(f"Number of documents with a hygiene score of 20: {document_count}")
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of documents with a hygiene score of 20: 82
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66c5683568cb6345c74bc03e'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 

In [21]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
results_df = pd.DataFrame(list(results))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {results_df.shape[0]}")
# Display the first 10 rows of the DataFrame
print(results_df.head(10))

Number of rows in the DataFrame: 82
                        _id   FHRSID  ChangesByServerID  \
0  66c5683568cb6345c74bc03e   110681                  0   
1  66c5683568cb6345c74bc3be   612039                  0   
2  66c5683568cb6345c74bc6d2   730933                  0   
3  66c5683568cb6345c74bc8be   172735                  0   
4  66c5683568cb6345c74bc8ca   172953                  0   
5  66c5683568cb6345c74bd267   512854                  0   
6  66c5683568cb6345c74bd499  1537089                  0   
7  66c5683568cb6345c74be9b6   155648                  0   
8  66c5683668cb6345c74bedf9  1012883                  0   
9  66c5683668cb6345c74bf60e   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [22]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}
# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print(f"Number of documents with London as Local Authority and RatingValue >= 4: {document_count}")
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of documents with London as Local Authority and RatingValue >= 4: 0
None


In [23]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
results_df = pd.DataFrame(list(results))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {results_df.shape[0]}")
# Display the first 10 rows of the DataFrame
print(results_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [24]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074
longitude = -0.1278

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}
sort = [("scores.Hygiene", 1)]
limit = 10

# Retrieve the results
results = establishments.find(query).sort(sort).limit(limit)

# Print the results
for result in results:
    pprint(result)


In [25]:
# Convert result to Pandas DataFrame
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {results_df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(results_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [26]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
# Execute the pipeline
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(f"Number of documents in the result: {len(results)}")
# Print the first 10 results
pprint(results[:10])

Number of documents in the result: 55
[{'_id': 'Thanet', 'count': 2260},
 {'_id': 'Greenwich', 'count': 1764},
 {'_id': 'Maidstone', 'count': 1426},
 {'_id': 'Newham', 'count': 1422},
 {'_id': 'Swale', 'count': 1372},
 {'_id': 'Chelmsford', 'count': 1360},
 {'_id': 'Medway', 'count': 1344},
 {'_id': 'Bexley', 'count': 1214},
 {'_id': 'Southend-On-Sea', 'count': 1172},
 {'_id': 'Tendring', 'count': 1084}]


In [27]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {results_df.shape[0]}")
# Display the first 10 rows of the DataFrame
print(results_df.head(10))

Number of rows in the DataFrame: 55
               _id  count
0           Thanet   2260
1        Greenwich   1764
2        Maidstone   1426
3           Newham   1422
4            Swale   1372
5       Chelmsford   1360
6           Medway   1344
7           Bexley   1214
8  Southend-On-Sea   1172
9         Tendring   1084
